In [ ]:
from jupyter_bbox_widget import BBoxWidget
import ipywidgets as widgets
import os
import json
import base64

In [ ]:
def encode_image(filepath):
    with open(filepath, 'rb') as f:
        image_bytes = f.read()
    encoded = str(base64.b64encode(image_bytes), 'utf-8')
    return "data:image/jpg;base64,"+encoded

In [ ]:
CLASS_LABELS = ['apple', 'orange', 'pear']

UNLABELLED_DATA_DIR = '/nvdli-nano/data/object-detection/custom/raw_images'
files = sorted(os.listdir(UNLABELLED_DATA_DIR))

annotations = {}
annotations_path = f'{UNLABELLED_DATA_DIR}/annotations.json'

In [ ]:
# a progress bar to show how far we got
w_progress = widgets.IntProgress(value=0, max=len(files), description='Progress')
# the bbox widget
w_bbox = BBoxWidget(
    image = encode_image(os.path.join(UNLABELLED_DATA_DIR, files[0])),
    classes=CLASS_LABELS
)

# combine widgets into a container
w_container = widgets.VBox([
    w_progress,
    w_bbox,
])

In [ ]:
# when Skip button is pressed we move on to the next file
@w_bbox.on_skip
def skip():
    w_progress.value += 1
    # open new image in the widget
    image_file = files[w_progress.value]
    w_bbox.image = encode_image(os.path.join(UNLABELLED_DATA_DIR, image_file))
    # here we assign an empty list to bboxes but 
    # we could also run a detection model on the file
    # and use its output for creating inital bboxes
    w_bbox.bboxes = [] 

# when Submit button is pressed we save current annotations
# and then move on to the next file
@w_bbox.on_submit
def submit():
    image_file = files[w_progress.value]
    # save annotations for current image
    annotations[image_file] = w_bbox.bboxes
    with open(annotations_path, 'w') as f:
        json.dump(annotations, f, indent=4)
    # move on to the next file
    skip()

In [ ]:
w_container